In [1]:
import cv2
import os

import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_face_mesh = mp.solutions.face_mesh

In [2]:
dirs = ["../data/" + k + "/" for k in os.listdir("../data/")]
dirs[:5]

['../data/anger/',
 '../data/contempt/',
 '../data/disgust/',
 '../data/fear/',
 '../data/happy/']

In [3]:
IMAGE_FILES = []

for d in dirs:
    IMAGE_FILES += [d+k for k in os.listdir(d)]

IMAGE_FILES[:3], len(IMAGE_FILES)

(['../data/anger/S010_004_00000017.png',
  '../data/anger/S010_004_00000018.png',
  '../data/anger/S010_004_00000019.png'],
 981)

In [4]:
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, refine_landmarks=True, min_detection_confidence=0.5)

In [5]:
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon

point = Point(0,0)
polygon = Polygon([(11,16),(15,15),(18,18)])
print(polygon.contains(point))

False


In [6]:
ncm_feat = {}

In [8]:
count = 0

import numpy as np

for idx, file in enumerate(IMAGE_FILES):
    image = cv2.imread(file)
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    
    if not results.multi_face_landmarks:
        continue
        
    annotated_image = image.copy()
    for face_landmarks in results.multi_face_landmarks:
        
        poly_indices = [            
            # lips
            [61, 185, 40, 39, 37, 0, 267, 269, 270, 409, 291,375,321,405,314,17,84,181,91,146],
            
            # nose
            [1,2,98,168],
            
            #rightEye
            [247, 30, 29, 27, 28, 56, 190,243,112,26,22,23,24,110,25,130],
            
            #rightEyebrow
            [247, 30, 29, 27, 28, 56, 190,193,55,107,66,105,63,70,156],
            
            [291,375,321,405,314,17,84,181,176,148,152,377,400,378,379,365,397],
            
            [327,454,323,291],
            
            [130,50,93,234],
            
        ]
        
        ncm_feat[file] = np.array([])
        
        c = 0
        for p in poly_indices:
            annotated_image = image.copy()
            
            allowedPoints = []
            polygon = []
            for i in p:
                k = face_landmarks.landmark[i]
                x,y,z = k.x,k.y,k.z
                x = int(annotated_image.shape[0]*x)
                y = int(annotated_image.shape[1]*y)
                polygon.append((x,y))
                allowedPoints.append([x,y])
                
            polygon = Polygon(polygon)
                
            for i in range(annotated_image.shape[0]):
                for j in range(annotated_image.shape[1]):
                    if polygon.contains(Point(i,j)):
                        allowedPoints.append([i,j])
                        
            for i in range(annotated_image.shape[0]):
                for j in range(annotated_image.shape[1]):
                    if [i,j] not in allowedPoints:
                        annotated_image[j][i] = 0
                    
            gray = cv2.cvtColor(annotated_image, cv2.COLOR_BGR2GRAY)
        
            M = np.zeros((3,3))
            
            for i in range(3):
                for j in range(3):
                    for k in range(gray.shape[0]):
                        for l in range(gray.shape[1]):
                            M[i][j] += pow(k,i)*pow(l,j)*gray[k][l]
                            
            xc = M[1][0]/M[0][0]
            yc = M[0][1]/M[0][0]
            
            mu = np.zeros((3,3))
            
            for i in range(3):
                for j in range(3):
                    for k in range(gray.shape[0]):
                        for l in range(gray.shape[1]):
                            mu[i][j] += pow(k-xc,i)*pow(l-yc,j)*gray[k][l]
            
            nu = np.zeros((3,3))
            
            for i in range(3):
                for j in range(3):
                    nu[i][j] = mu[i][j]/pow(M[0][0], (i+j)/3)
                    
            ncm_feat[file] = np.append(ncm_feat[file], nu.flatten(), axis=0)
            
        
    count += 1
    if count%10 == 0:
        print(count)
        
#         cv2.imwrite("../data/anger/" + str(idx) + '_removed.png', annotated_image)
        
#     break

10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980


In [9]:
len(ncm_feat)

981

In [10]:
import h5py

f = h5py.File("ncm_feat.h5", "w")

for k in ncm_feat:
    f[k.replace("/","__")] = ncm_feat[k]

f.close()

In [11]:
s = []

for k in ncm_feat:
    s.append(ncm_feat[k].shape)
    
list(set(s))

[(63,)]

In [13]:
ncm = {}

with h5py.File("ncm_feat.h5", "r") as f:
    for k in list(f.keys()):
        ncm[k] = np.array(f[k])
        
len(ncm)

981